In [2]:
# Load packages
import pandas as pd
import numpy as np
from statistics import *
from scipy.stats import *
from math import *
import statsmodels.api as sm

# Load the data
data_Index = pd.read_excel("/files/exercises/Homeworks/HW4/TP4.xls", sheet_name='S&P100Index', skiprows=3)
data_Constituents = pd.read_excel("/files/exercises/Homeworks/HW4/TP4.xls", sheet_name='S&P100Constituents', skiprows=[0,1,2,4])
data_Tbill = pd.read_excel("/files/exercises/Homeworks/HW4/TP4.xls", sheet_name='TBill3Months', skiprows=[0,1,2,4])


In [3]:
print(data_Constituents)

          Name   AES  ALCOA  ALLEGHENY TECHS.  AMER.ELEC.PWR.  AMER.EXPRESS  \
0   1992-11-12  3.48   8.48             29.50           30.75          6.80   
1   1992-11-19  3.72   8.19             29.62           31.25          6.96   
2   1992-11-26  4.09   8.52             31.10           31.13          7.27   
3   1992-12-03  3.88   8.75             32.95           31.25          7.23   
4   1992-12-10  4.05   9.02             32.58           31.63          7.70   
..         ...   ...    ...               ...             ...           ...   
518 2002-10-17  1.05  22.50              6.65           18.36         34.10   
519 2002-10-24  1.28  22.69              7.48           21.87         32.56   
520 2002-10-31  1.77  22.06              6.85           25.64         36.37   
521 2002-11-07  1.92  23.47              6.59           26.56         35.50   
522 2002-11-14  1.53  22.35              6.21           25.99         35.20   

     AMER.INTL.GP.  AMGEN  ANHEUSER - BUSCH  AOL TI

In [4]:
print(data_Index)

                    Name S&P 100  -  PRICE INDEX  \
0                   Code             S&P100I(PI)   
1    1992-11-12 00:00:00                  192.13   
2    1992-11-19 00:00:00                  192.31   
3    1992-11-26 00:00:00                  195.36   
4    1992-12-03 00:00:00                   195.4   
..                   ...                     ...   
519  2002-10-17 00:00:00                  446.21   
520  2002-10-24 00:00:00                  447.88   
521  2002-10-31 00:00:00                   451.2   
522  2002-11-07 00:00:00                  460.55   
523  2002-11-14 00:00:00                  450.61   

    S&P 500 COMPOSITE  -  PRICE INDEX  
0                         S&PCOMP(PI)  
1                              422.88  
2                              423.61  
3                              429.19  
4                              429.92  
..                                ...  
519                             879.2  
520                             882.5  
521            

## Test of the CAPM: Time-Series Approach 
### 1.  The standard Sharpe-Lintner CAPM assumes the existence of a constant risk-free rate and is written as:E[ri] =rf+βi(E[rm]−rf). For each stock, estimate the betas on the 12/11/1992–15/8/2001 period, using the SP500 as a market proxy.  Issue:  how can you form the expectations?

In [5]:
# Compute the returns for S&P500
nb_ret = len(data_Index.iloc[:,1]) - 1
RM = np.array(nb_ret)
RM = (data_Index.iloc[2:len(data_Index.iloc[:,1])-66, 2].values-data_Index.iloc[1:len(data_Index.iloc[:,1])-67, 2].values)/data_Index.iloc[1:len(data_Index.iloc[:,1])-67, 2].values
beta = np.zeros(len(data_Constituents.columns)-3, dtype = float)

for i in range (1,len(data_Constituents.columns)-2):
    # Compute the return for each stock
    RA = np.array(nb_ret)
    RA = (data_Constituents.iloc[1:len(data_Index.iloc[:,1])-67, i].values-data_Constituents.iloc[0:len(data_Index.iloc[:,1])-68, i].values)/data_Constituents.iloc[0:len(data_Index.iloc[:,1])-68, i].values

    # Compute beta
    
    alpha = np.cov(RM.astype(float),RA)/np.var(RM)
    beta[i-1] = alpha[0][1]
beta    

array([1.01505432, 0.6182784 , 0.70380973, 0.26346238, 1.50441926,
       1.19673722, 1.01216017, 0.4291283 , 1.88455388, 0.90922178,
       0.65722592, 0.62443455, 1.22226783, 1.21338441, 0.69891251,
       0.99125976, 0.70120313, 0.91925168, 0.63746671, 0.67110961,
       0.46723244, 0.75166534, 1.72654285, 1.82707304, 1.17373027,
       0.74277935, 0.85522308, 1.13837317, 1.02011609, 0.60863187,
       0.77934175, 0.59685211, 0.50890523, 1.80003467, 0.16673504,
       0.20008449, 0.41517969, 1.15327501, 0.82517261, 0.5120496 ,
       1.25926839, 0.90653964, 0.66215846, 0.85007006, 1.04319166,
       0.80922628, 0.37298608, 1.34878146, 1.39826269, 1.09543155,
       1.30640239, 0.9937648 , 0.63576365, 1.52944417, 0.67760012,
       0.86228733, 0.91813362, 0.68493249, 1.17030478, 1.11640512,
       0.725537  , 1.89112067, 1.21444974, 1.32908818, 1.98338597,
       0.71777045, 1.57653744, 0.56293924, 0.88575772, 0.75026064,
       0.50151123, 0.50054634, 1.41119172, 0.50145047, 0.85988

### 2.  A first method to test the CAPM is based on the following time-series regres-sion:zi,t=αi+βizm,t+εi,t,wherezi,t=ri,t−rf,t,  andεi,tis  a  zero  mean  noise.   The  null  hypothesisH0:αi=  0  can  then  be  tested  for  each  asset  separately  using  standardt-statistics.Using the remaining data,  perform this na ̈ıve test of the CAPM and checkhow well the market explains individual stock returns.

In [29]:
# Compute z
rf = np.array(nb_ret)
zm = np.array(nb_ret)
zi = np.array(nb_ret)
betas = np.zeros(len(data_Constituents.columns)-2)
alphas = np.zeros(len(data_Constituents.columns)-2)
RF = (data_Tbill.iloc[1:len(data_Index.iloc[:,1])-67, 1].values - data_Tbill.iloc[0:len(data_Index.iloc[:,1])-68, 1].values)/data_Tbill.iloc[0:len(data_Index.iloc[:,1])-68, 1].values
zm = RM - RF

for i in range(1, len(data_Constituents.columns)-2):
    zi = (data_Constituents.iloc[1:len(data_Index.iloc[:,1])-67, i].values-data_Constituents.iloc[0:len(data_Index.iloc[:,1])-68, i].values)/data_Constituents.iloc[0:len(data_Index.iloc[:,1])-68, i].values - RF


    # Linear regression
    independent_vars = zm.astype('float64')
    X = sm.add_constant(independent_vars)
    y =  zi

    reg1 = sm.OLS(endog = y, exog = X, missing='drop')
    results = reg1.fit()
   # betas[i] = results.params['rm_rf']
   # alphas[i] = results.params['const']
    print("\n", "\n", "Company", i)
    print(results.summary())


 
 Company 1
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.232
Model:                            OLS   Adj. R-squared:                  0.230
Method:                 Least Squares   F-statistic:                     137.0
Date:                Sun, 11 Oct 2020   Prob (F-statistic):           7.77e-28
Time:                        11:18:34   Log-Likelihood:                 675.24
No. Observations:                 456   AIC:                            -1346.
Df Residuals:                     454   BIC:                            -1338.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0043      0.003      